In [49]:
import pandas as pd
import numpy as np
import RNG

### Simulation parameters and helper functions

In [50]:
num_order = np.random.randint(low=1,high=11,size=1)[0]#  ~ Unif[1,11)

# unit of time: minuate
spawn_meant = 0.25
teleport_meant = 0.5
assemble_meant = 1.0
insp_meant = 1.5
pass_meant = 1.0

class ActionLogger(object):
    def __init__(self):
        self.clock = 0.0
        self.alog = []
        self.tlog = []

    def write_act(self,a,t):
        self.clock += t
        self.alog.append(a)
        self.tlog.append(self.clock)

### Station 1 - Wheelsets x 2

In [51]:
logger1 = ActionLogger()
for _ in range(num_order):
    for _ in range(2):
        # sourcing
        for _ in range(7):
            logger1.write_act("SpawnPart",RNG.Expon(spawn_meant,1))

        # manufacturing
        # (rim + tire) x 2
        for _ in range(2):
            logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # rim
            logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # tire
            logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # rim + tire

        # (wheels + axle)
        logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # axle
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + wheel 1
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + wheel 2
        
        # (axle + axle)
        logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # connecting plate
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + axle

        # inspection
        logger1.write_act("Inspection",RNG.Expon(insp_meant,1))
    # pass wheelsets to downstream
    logger1.write_act("Pass",RNG.Expon(pass_meant,1))

### Station 2 - Chassis

In [52]:
upstream_parts_idx = np.where(np.array(logger1.alog)=="Pass")[0]
upstream_parts_idx

array([ 38,  77, 116, 155, 194, 233, 272], dtype=int64)

In [53]:
upstream_parts_time = np.array(logger1.tlog)[upstream_parts_idx]
upstream_parts_time

array([ 27.78306928,  48.13629974,  73.06424703,  99.2046338 ,
       126.06710166, 142.56458032, 162.07319741])

In [55]:
upstream_parts_idx = np.where(np.array(logger1.alog)=="Pass")[0]
upstream_parts_time = np.array(logger1.tlog)[upstream_parts_idx]

logger2 = ActionLogger()
for i in range(num_order):
    # sourcing
    for _ in range(9):
        logger2.write_act("SpawnPart",RNG.Expon(spawn_meant,2))

    # manufacturing
    # at least need one bottom plate
    # (long plate + long plate)
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

    # need to receive wheelsets from the upstream
    if logger2.clock < upstream_parts_time[i]:
        # wait (may have plating assembled rather than just waiting)
        logger2.clock = upstream_parts_time[i]

    # (connected plate + wheelsets)
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

    # (front bumper 1)
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

    # (front bumper 2)
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

    # (chassis + front bumper)
    logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

    # inspection & pass chassis to downstream
    logger2.write_act("Inspection",RNG.Expon(insp_meant,2))
    logger2.write_act("Pass",RNG.Expon(pass_meant,2))

In [56]:
logger2.alog[:15]

['SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'SpawnPart',
 'TeleportPart',
 'TeleportPart',
 'Assemble',
 'TeleportPart',
 'Assemble',
 'TeleportPart']

In [57]:
logger2.tlog[:15]

[0.4611444052785237,
 0.47781568736914204,
 0.5759271421370202,
 0.8793353186476065,
 0.9648705265461307,
 1.3947191561010528,
 1.7964806362277375,
 1.92925191118627,
 1.9451268478314465,
 2.340645431582912,
 2.3470037515323665,
 2.420578734297464,
 27.929316610614165,
 29.111904250827816,
 30.008574192269194]

### Station 3 - Windshield & front body

In [ ]:
logger3 = ActionLogger()

### Station 4 - Ceiling & rear body

In [ ]:
logger4 = ActionLogger()